#### A. Configuración General.

In [ ]:
#1. Librerías.
%run "../librerias.ipynb"

In [ ]:
#2. Constantes.
#a. Constantes generales.
%run "../constantes.ipynb"

#b. Constantes a definir por el usuari.
#i. Dataset de lectura (post-feature engineering).
dataset_con_fe = dataset_file_fe_12_3
#ii. Información sobre rango temporal del modelo.
cantidad_meses_train = 1
ventana = 3
#iii. Meses de train y test.
mes_train = 202106
mes_test = mes_test
#iv. Ruta del modelo guardado post-optimización de hiperparámetros.
modelo =  "{}".format(modelos_path) + 'rf_{}_{}.sav'.format(cantidad_meses_train,ventana)
#v. Ruta de la BBDD donde se almacenan los hiperparámetros óptimos post-Optuna.
storage_name = "sqlite:///" + db_path + "optimization_tree.db"
study_name = f"exp_rf_{cantidad_meses_train}_{ventana}"

In [ ]:
#3. Lectura de datos.
data = pd.read_parquet(dataset_con_fe)

In [ ]:
#4. Importo el modelo.
model_rf = pickle.load(open(modelo, 'rb'))

#### B. Pre-procesamiento.

In [ ]:
#1. Dividimos entre conjuntos de datos entre Train y Test.
#a. Train.
X = data[data['foto_mes'] == mes_train]
y = X['clase_ternaria']
X = X.drop(columns=['clase_ternaria'])

#b. Test.
X_futuro = data[data['foto_mes'] == mes_test]
y_futuro = X_futuro['clase_ternaria']
X_futuro = X_futuro.drop(columns=['clase_ternaria'])

#### C. Envíos para encontrar el punto de corte óptimo con 10 semillas.

In [ ]:
#1. Configuramos la cantidad de veces que vamos a subir cada curva de intentos (semillas).
cantidad_semillas = 10
semillas = [np.random.randint(0, 10000) for _ in range(cantidad_semillas)]

In [ ]:
#2. Entrenamos y predecimos Agosto.
#i. Recorremos diferentes semillas para entrenar.
i= 0
for semilla in semillas:
    print("\n---------------------------------------")
    print("Semilla: {}.".format(semilla))
    #ii. Activamos la API para realizar envíos a Kaggle.
    api = KaggleApi()
    api.authenticate()
    #iii. Entrenamos el modelo Random Forest con los mejores hiperparámetros y la semilla aleatoria.
    print("--> Comienza el entrenamiento...")
    model_rf = RandomForestClassifier(
        n_estimators=1000,
        **best_params,  # Usa los mejores hiperparámetros obtenidos de Optuna
        max_samples=0.7,
        random_state=semilla,  # Cambia la semilla aquí para cada iteración
        n_jobs=-1,
        oob_score=True
    )
    
    model_rf.fit(X, y)
    #iv. Predecimos Agosto.
    predicciones = model_rf.predict_proba(X_futuro)[:, 1]  # Probabilidad de ser "BAJA+2"
    #v. Le pegamos la probabilidad de ser "BAJA" a cada cliente.
    X_futuro['Probabilidad'] = predicciones
    #vi. Ordenamos a los clientes por probabilidad de ser "BAJA" de forma descendente.
    tb_entrega = X_futuro.sort_values(by='Probabilidad', ascending=False)
    #vii. Genero una lista de distintos cortes candidatos, para enviar a Kaggle.
    cortes = range(8500,13500,500)
    #viii. Generamos las distintas predicciones de clases a partir de los distintos cortes posibles.
    num_subida_kaggle = 1
    print("--> Comienzan los envíos a Kaggle...\n")
    for envios in cortes:
        #1. Le ponemos clase 1 ("BAJA") a los primeros "envios" con mayor probabilidad.
        tb_entrega['Predicted'] = 0
        tb_entrega.iloc[:envios, tb_entrega.columns.get_loc('Predicted')] = 1
        resultados = tb_entrega[["numero_de_cliente", 'Predicted']].reset_index(drop=True)
        
        print("Cantidad de clientes {}".format(envios))
        #2. Guardamos el archivo para Kaggle.
        nombre_archivo = "KRF_{}_{}_EK_0{}_{}.csv".format(cantidad_meses_train,ventana,num_subida_kaggle,semilla) #DF con -X meses, Y ventana, número de intento, número de semilla.
        ruta_archivo= "{}/{}".format(exp_path,nombre_archivo)
        resultados.to_csv(ruta_archivo, index=False)
        
        num_subida_kaggle += 1
        i += 1
        
        #3. Envío a Kaggle.
        #a. Defino los parámetros claves.
        mensaje = f'Archivo {nombre_archivo}.RF Train {cantidad_meses_train}. Semilla:{semilla},corte:{envios}' # Datos de Train, si está con Data Drifting o no, semilla y punto de corte de prueba.
        competencia = 'dm-ey-f-2024-segunda'
        #c. Subo la Submission.
        while i<=18:
                api.competition_submit(file_name=ruta_archivo, message=mensaje, competition=competencia)
                print("Submission successful!")
                break
        else:
            print("Esperamos 30 segundos...")
            time.sleep(30)
            api.competition_submit(file_name=ruta_archivo, message=mensaje, competition=competencia)
            print("Submission successful!")
            i= 0
        
    #d. Borramos la columna de predicciones anterior.
    X_futuro.drop(["Probabilidad"],axis=1,inplace=True)